In [ ]:
import sdmx
import pandas as pd

nome_base = 'BOP'

# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]


indicadores = base.codelist['CL_PCPS_INDICATOR']
indicadores_commodities = indicadores.items

params = dict(
    startPeriod="2000",
)


In [ ]:
# Kit essencial (indicador, unidade)
core_indicators = [
    ("CAB", "USDB"),   # Current account balance
    ("GS", "USDB"),    # Goods & services (saldo)
    ("G", "USDB"),     # Goods (saldo)
    ("S", "USDB"),     # Services (saldo)
    ("IN1", "USDB"),   # Primary income
    ("IN2", "USDB"),   # Secondary income
    ("NLNB", "USDB"),  # Net lending/borrowing
    ("KAB", "USDB"),   # Capital account balance
    ("D", "USDB"),     # Direct investment (conta financeira)
    ("P", "USDB"),     # Portfolio investment
    ("O", "USDB"),     # Other investment
    ("FAB", "USDB"),   # Financial account balance
    ("R", "USDB"),     # Reserve assets (total)
    ("R_F2", "USDB"),  # Reservas: moeda e depósitos
    ("R_F3", "USDB"),  # Reservas: títulos de dívida
    ("R_F11", "USDB"), # Reservas: ouro
    ("R_F12", "USDB"), # Reservas: SDR
    ("NIIP", "USDB"),  # Net International Investment Position
    ("IIP", "USDB"),   # International Investment Position (estoque total)
    ("EO", "USDB"),    # Net errors and omissions
]



In [ ]:
indicadores_account_entry = [
    # — Conta corrente e capital: use o SALDO pronto (NET = crédito – débito)
    ("CAB",   "NETCD_T"),  # Current account balance
    ("KAB",   "NETCD_T"),  # Capital account balance
    ("GS",    "NETCD_T"),  # Goods & services (saldo)
    ("G",     "NETCD_T"),  # Goods (saldo)
    ("S",     "NETCD_T"),  # Services (saldo)
    ("IN1",   "NETCD_T"),  # Primary income (saldo)
    ("IN2",   "NETCD_T"),  # Secondary income (saldo)
    ("NLNB",  "NETCD_T"),  # Net lending/borrowing (saldo)

    # — Conta financeira (fluxos): pegue ativos, passivos e o net
    ("D",     "A_NFA_T"),      # Direct investment – ativos
    ("D",     "L_NIL_T"),      # Direct investment – passivos
    ("D",     "NNAFANIL_T"),   # Direct investment – net

    ("P",     "A_NFA_T"),      # Portfolio investment – ativos
    ("P",     "L_NIL_T"),      # Portfolio investment – passivos
    ("P",     "NNAFANIL_T"),   # Portfolio investment – net

    ("O",     "A_NFA_T"),      # Other investment – ativos
    ("O",     "L_NIL_T"),      # Other investment – passivos
    ("O",     "NNAFANIL_T"),   # Other investment – net

    ("R",     "A_NFA_T"),      # Reserve assets – ativos
    ("R",     "L_NIL_T"),      # Reserve assets – passivos (geralmente zero/incomum, mas deixei para consistência)
    ("R",     "NNAFANIL_T"),   # Reserve assets – net

    # Agregado da conta financeira já “netado”
    ("FAB",   "NNAFANIL_T"),   # Financial account balance (assets – liabilities)

    # — Posições (estoques): International Investment Position
    ("IIP",   "A_P"),          # Ativos externos (posição)
    ("IIP",   "L_P"),          # Passivos externos (posição)
    ("IIP",   "NETAL_P"),      # NIIP via IIP (ativos – passivos)

    # NIIP (Net International Investment Position) direto, se existir na base
    ("NIIP",  "NETAL_P"),
]


In [ ]:
df_final = pd.DataFrame() 

for i,indicador in enumerate(indicadores_commodities):

    # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
    key = dict(
        COUNTRY = 
        BOP_ACCOUNTING_ENTRY = 
        INDICATOR = 
        FREQUENCY = 
    )
    # Seleção de chave
    # key = dict(
    #     COUNTRY=["BRA", "USA"],
    #     INDICATOR=["FL_S13_POGDP_PT", "FL_PS_POGDP_PT"],
    #     FREQUENCY="A"
    # )

    # Baixar os dados
    data_msg = client.data("PCPS", key=key, params=params, dsd=estrutura)

    # Converter para pandas DataFrame
    df = sdmx.to_pandas(
        data_msg, 
        #datetime="TIME_PERIOD"
        )

    df = df.reset_index()

    df_final = pd.concat([df_final, df], ignore_index=True)
    print(f"Finalizada etapa {i+1}")




display(df_final.head())

df_final.to_csv("bases_dados/commodities.csv", index=False)

# Teste extra-oficial

In [11]:
import sdmx
import pandas as pd

nome_base = 'BOP'
pais = 'RUS'
# key = 'USA+BRA.G63G_S13_POFYGDP.A'
key = dict(
    COUNTRY = pais,
    # BOP_ACCOUNTING_ENTRY = "NETCD_T",
    # INDICATOR = "CAB",
    # FREQUENCY = "A",
    # UNIT= "USDB"
)

# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

# Baixar os dados
data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

# Converter para pandas DataFrame
df = sdmx.to_pandas(
    data_msg, 
    #datetime="TIME_PERIOD"
    )

df = df.reset_index()

display(df.head())
df.to_csv(f'../Resultados/{nome_base}_{pais}.csv', index=False)

,COUNTRY,BOP_ACCOUNTING_ENTRY,INDICATOR,UNIT,FREQUENCY,TIME_PERIOD,value
0,RUS,A_NFA_T,D1_F3,USD,A,2014,0.000000e+00
1,RUS,A_NFA_T,D1_F3,USD,A,2015,-4.098000e+07
2,RUS,A_NFA_T,D1_F3,USD,A,2016,1.279190e+09
3,RUS,A_NFA_T,D1_F3,USD,A,2017,-3.674900e+08
4,RUS,A_NFA_T,D1_F3,USD,A,2018,7.147300e+08


In [10]:
df.describe(exclude="number")

,COUNTRY,BOP_ACCOUNTING_ENTRY,INDICATOR,UNIT,FREQUENCY,TIME_PERIOD
count,46996,46996,46996,46996,46996,46996
unique,1,9,261,1,2,126
top,IND,NETCD_T,O_F4,USD,Q,2010
freq,46996,10787,378,46996,37642,442


In [19]:
core_indicators = [
    "CAB",   # Current account balance
    "GS",    # Goods & services (saldo)
    "G",     # Goods (saldo)
    "S",     # Services (saldo)
    "IN1",   # Primary income
    "IN2",   # Secondary income
    "NLNB",  # Net lending/borrowing
    "KAB",   # Capital account balance
    "D",     # Direct investment (conta financeira)
    "P",     # Portfolio investment
    "O",     # Other investment
    "FAB",   # Financial account balance
    "R",     # Reserve assets (total)
    "R_F2",  # Reservas: moeda e depósitos
    "R_F3",  # Reservas: títulos de dívida
    "R_F11", # Reservas: ouro
    "R_F12", # Reservas: SDR
    "NIIP",  # Net International Investment Position
    "IIP",   # International Investment Position (estoque total)
    "EO",    # Net errors and omissions
]

In [25]:
important_countries = [
    # Maiores economias / benchmarks
    "USA","CHN","DEU","JPN","GBR","FRA","ITA","IND","CAN","AUS","KOR","ESP","NLD",
    # Europa ampliada
    "IRL","CHE","LUX","SWE","DNK","FIN","POL","AUT","BEL","CZE","HUN","ROU","PRT","GRC",
    # Américas (desenvolvidos + EM)
    "BRA","MEX","ARG","COL","CHL","PER","URY",
    # Ásia emergente e manufatura
    "TWN","SGP","HKG","THA","MYS","VNM","IDN","PHL",
    # Oriente Médio / petróleo
    "SAU","ARE","QAT","KWT","IRN","ISR",
    # Commodities avançados / energia
    "NOR","RUS",
    # África (representativos)
    "ZAF","EGY","NGA","GHA","KEN",
]


In [27]:
# indicadores = base.codelist['CL_BOP_ACCOUNTING_ENTRY']
# indicadores_commodities = indicadores.items

lista_accounting = [
    "CD_T" ,
    "DB_T" ,
    "NETCD_T",
    "A_NFA_T" ,
    "L_NIL_T" ,
    "NNAFANIL_T" ,
    "A_P" , 
    "L_P" ,
    "NETAL_P" 
]


import sdmx
import pandas as pd

nome_base = 'BOP'


# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

df_final = pd.DataFrame() 


# Abrir o arquivo para adicionar conteúdo (append)
with open("1_DICIONARIO.txt", "a", encoding="utf-8") as f:

    for country in important_countries:
        f.write("----------------------\n\n")
        for core_indicator in core_indicators:
            for i,ac in enumerate(lista_accounting):

                # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
                key = dict(
                    COUNTRY = country,
                    BOP_ACCOUNTING_ENTRY = ac,
                    INDICATOR = core_indicator,
                    FREQUENCY = "A",
                    # UNIT= "USDB"
                )

                # Baixar os dados
                data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

                # Converter para pandas DataFrame
                df = sdmx.to_pandas(
                    data_msg, 
                    #datetime="TIME_PERIOD"
                    )

                df = df.reset_index()

                df_final = pd.concat([df_final, df], ignore_index=True)
                if not df.empty:
                    print(f"País {country},indicador: {core_indicator}, accounting: {ac}")
                    f.write(f"País {country},indicador: {core_indicator}, accounting: {ac}")
                print(f"Finalizada etapa {i}") if i == 1 else print




display(df_final.head())

Finalizada etapa 1
País USA,indicador: CAB, accounting: NETCD_T
País USA,indicador: GS, accounting: CD_T
País USA,indicador: GS, accounting: DB_T
Finalizada etapa 1
País USA,indicador: GS, accounting: NETCD_T
País USA,indicador: G, accounting: CD_T
País USA,indicador: G, accounting: DB_T
Finalizada etapa 1
País USA,indicador: G, accounting: NETCD_T
País USA,indicador: S, accounting: CD_T
País USA,indicador: S, accounting: DB_T
Finalizada etapa 1
País USA,indicador: S, accounting: NETCD_T
País USA,indicador: IN1, accounting: CD_T
País USA,indicador: IN1, accounting: DB_T
Finalizada etapa 1
País USA,indicador: IN1, accounting: NETCD_T
País USA,indicador: IN2, accounting: CD_T
País USA,indicador: IN2, accounting: DB_T
Finalizada etapa 1
País USA,indicador: IN2, accounting: NETCD_T
Finalizada etapa 1
Finalizada etapa 1
País USA,indicador: KAB, accounting: NETCD_T
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
Finalizada etapa 1
País USA,indicador: FAB, accounting: NNAFANIL_T
Fina

,index,COUNTRY,BOP_ACCOUNTING_ENTRY,INDICATOR,UNIT,FREQUENCY,TIME_PERIOD,value
0,NaN,USA,NETCD_T,CAB,USD,A,2000,-4.019280e+11
1,NaN,USA,NETCD_T,CAB,USD,A,2001,-3.940790e+11
2,NaN,USA,NETCD_T,CAB,USD,A,2002,-4.561090e+11
3,NaN,USA,NETCD_T,CAB,USD,A,2003,-5.222970e+11
4,NaN,USA,NETCD_T,CAB,USD,A,2004,-6.358870e+11


In [30]:
df_final.to_csv("../Resultados/BOP.csv", index=False)

In [13]:
core_indicators = [
    "CAB", "CABXEF", "KAB", "FAB", "FABXRRI", "EO",
    "SF", "GS", "IN1", "IN2",
    "DXEF", "D_F5", "D_FL",
    "PXEF", "P_F5", "P_F3",
    "O_F4", "O_F2", "O_F81",
    "RUE", "R_F",
]

In [14]:
important_countries = [
    # Maiores economias / benchmarks
    "USA","CHN","DEU","JPN","GBR","FRA","ITA","IND","CAN","AUS","KOR","ESP","NLD",
    # Europa ampliada
    "IRL","CHE","LUX","SWE","DNK","FIN","POL","AUT","BEL","CZE","HUN","ROU","PRT","GRC",
    # Américas (desenvolvidos + EM)
    "BRA","MEX","ARG","COL","CHL","PER","URY",
    # Ásia emergente e manufatura
    "TWN","SGP","HKG","THA","MYS","VNM","IDN","PHL",
    # Oriente Médio / petróleo
    "SAU","ARE","QAT","KWT","IRN","ISR",
    # Commodities avançados / energia
    "NOR","RUS",
    # África (representativos)
    "ZAF","EGY","NGA","GHA","KEN",
]

In [15]:
import sdmx
import pandas as pd

nome_base = 'BOP'


# Cliente IMF
client = sdmx.Client("IMF_DATA")
base = client.dataflow(nome_base)
nome_estrutura = list(base.structure.keys())[0]
estrutura = base.structure[nome_estrutura]

params = dict(
    startPeriod="2000",
)

df_final = pd.DataFrame() 


# Abrir o arquivo para adicionar conteúdo (append)
with open("1_DICIONARIO.txt", "a", encoding="utf-8") as f:

    for country in important_countries:
        f.write("----------------------\n\n")
        for i,core_indicator in enumerate(core_indicators):
            # key = f'BRA+USA+CHN+IND.{indicador}.USD.A'
            key = dict(
                COUNTRY = country,
                # BOP_ACCOUNTING_ENTRY = ac,
                INDICATOR = core_indicator,
                FREQUENCY = "Q",
                # UNIT= "USDB"
            )

            # Baixar os dados
            data_msg = client.data(nome_base, key=key, params=params, dsd=estrutura)

            # Converter para pandas DataFrame
            df = sdmx.to_pandas(
                data_msg, 
                #datetime="TIME_PERIOD"
                )

            df = df.reset_index()

            df_final = pd.concat([df_final, df], ignore_index=True)
            if not df.empty:
                print(f"País {country},indicador: {core_indicator}")
                f.write(f"\nPaís {country},indicador: {core_indicator}")
            print(f"Finalizada etapa {i}") if i == 1 else print




display(df_final.head())
df_final.to_csv("../Resultados/BOP.csv", index=False)

País USA,indicador: CAB
País USA,indicador: CABXEF
Finalizada etapa 1
País USA,indicador: KAB
País USA,indicador: FAB
País USA,indicador: FABXRRI
País USA,indicador: EO
País USA,indicador: SF
País USA,indicador: GS
País USA,indicador: IN1
País USA,indicador: IN2
País USA,indicador: DXEF
País USA,indicador: D_F5
País USA,indicador: D_FL
País USA,indicador: PXEF
País USA,indicador: P_F5
País USA,indicador: P_F3
País USA,indicador: O_F4
País USA,indicador: O_F2
País USA,indicador: O_F81
País USA,indicador: RUE
País USA,indicador: R_F
País CHN,indicador: CAB
País CHN,indicador: CABXEF
Finalizada etapa 1
País CHN,indicador: KAB
País CHN,indicador: FAB
País CHN,indicador: FABXRRI
País CHN,indicador: EO
País CHN,indicador: SF
País CHN,indicador: GS
País CHN,indicador: IN1
País CHN,indicador: IN2
País CHN,indicador: DXEF
País CHN,indicador: D_F5
País CHN,indicador: D_FL
País CHN,indicador: PXEF
País CHN,indicador: P_F5
País CHN,indicador: P_F3
País CHN,indicador: O_F4
País CHN,indicador: O_F2


,COUNTRY,BOP_ACCOUNTING_ENTRY,INDICATOR,UNIT,FREQUENCY,TIME_PERIOD,value,index
0,USA,NETCD_T,CAB,USD,Q,2000-Q1,-8.458500e+10,NaN
1,USA,NETCD_T,CAB,USD,Q,2000-Q2,-9.610400e+10,NaN
2,USA,NETCD_T,CAB,USD,Q,2000-Q3,-1.149650e+11,NaN
3,USA,NETCD_T,CAB,USD,Q,2000-Q4,-1.062740e+11,NaN
4,USA,NETCD_T,CAB,USD,Q,2001-Q1,-9.645400e+10,NaN
